In [1]:
import numpy as np

def interp_red(img, i, j):
    value = 0
    measured = 0
    if i % 2 == 1 and j % 2 == 1:
        # green2 -- use left and right values
        if j >= 1:
            value += img[i][j-1][0]
            measured += 1
        if (j+1) < img.shape[1]:
            value += img[i][j+1][0]
            measured += 1
    if i % 2 == 0 and j % 2 == 0:
        # green1 -- use top and bot values
        if i >= 1:
            value += img[i-1][j][0]
            measured += 1
        if (i+1) < img.shape[0]:
            value += img[i+1][j][0]
            measured += 1
    else:
        # blue -- use all diagonals
        if i >= 1 and j >=1:
            value += img[i-1][j-1][0]
            measured += 1
        if i >= 1 and (j+1) < img.shape[1]:
            value += img[i-1][j+1][0]
            measured += 1
        if (i+1) < img.shape[0] and j >= 1:
            value += img[i+1][j-1][0]
            measured += 1
        if (i+1) < img.shape[0] and (j+1) < img.shape[1]:
            value += img[i+1][j+1][0]
            measured += 1
    
    img[i][j][0] = value/measured
    
def interp_blue(img, i, j):
    value = 0
    measured = 0
    if i % 2 == 0 and j % 2 == 0:
        # green1 -- use left and right values
        if j >= 1:
            value += img[i][j-1][2]
            measured += 1
        if (j+1) < img.shape[1]:
            value += img[i][j+1][2]
            measured += 1
    if i % 2 == 1 and j % 2 == 1:
        # green2 -- use top and bot values
        if i >= 1:
            value += img[i-1][j][2]
            measured += 1
        if (i+1) < img.shape[0]:
            value += img[i+1][j][2]
            measured += 1
    else:
        # red -- use all diagonals
        if i >= 1 and j >=1:
            value += img[i-1][j-1][2]
            measured += 1
        if i >= 1 and (j+1) < img.shape[1]:
            value += img[i-1][j+1][2]
            measured += 1
        if (i+1) < img.shape[0] and j >= 1:
            value += img[i+1][j-1][2]
            measured += 1
        if (i+1) < img.shape[0] and (j+1) < img.shape[1]:
            value += img[i+1][j+1][2]
            measured += 1
    
    img[i][j][2] = value/measured

def interp_green(img, i, j):
    value = 0
    measured = 0
    
    if i >= 2:
        value += img[i-2][j][1]
        measured += 1
    if (i+2) < img.shape[0]:
        value += img[i+2][j][1]
        measured += 1
    if j >= 2:
        value += img[i][j-2][1]
        measured += 1
    if (j+2) < img.shape[1]:
        value += img[i][j+2][1]
        measured += 1
        
    img[i][j][1] = value/measured

def reinterp_bilinear_inefficient(img):
    # reinterpolate interpolated points with the bilinear algorithm
    # with the assumption of a bayer filter
    img = img.copy()
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if (i % 2 == 0 and j % 2 == 0) or (i % 2 == 1 and j % 2 == 1):
                # green was measured here
                interp_red(img, i, j)
                interp_blue(img, i, j)
            elif i % 2 == 0 and j % 2 == 1:
                # blue was measured here
                interp_red(img, i, j)
                interp_green(img, i, j)
            else:
                # red was measured here
                interp_green(img, i, j)
                interp_blue(img, i, j)
    return img

def reinterp_bilinear(img):
    img = img.astype('uint16').copy()
    
    red_blue_kernel = np.array([[1,2,1],[2,4,2],[1,2,1]]) / 4
    green_kernel = np.array([[0,1,0],[1,4,1],[0,1,0]]) / 4
    img[:,:,0] = cv2.filter2D(img[:,:,0], -1, red_blue_kernel)
    img[:,:,1] = cv2.filter2D(img[:,:,1], -1, green_kernel)
    img[:,:,2] = cv2.filter2D(img[:,:,2], -1, red_blue_kernel)
    
    return img

def reinterp_nearest_neighbor(img):
    # reinterpolate interpolated points with the nearest neighbor algorithm
    # with the assumption of bayer filter
    img = img.copy()
    for i in range(0, img.shape[0], 2):
        for j in range(0, img.shape[1], 2):
            # act in 2x2 squares
            
            # interpolate red
            img[i][j][0] = img[i+1][j][0]
            img[i][j+1][0] = img[i+1][j][0]
            img[i+1][j+1][0] = img[i+1][j][0]
            
            # interpolate green
            img[i][j+1][1] = img[i][j][1]
            img[i+1][j][1] = img[i+1][j+1][1]
            
            # interpolate blue
            img[i][j][2] = img[i][j+1][2]
            img[i+1][j][2] = img[i][j+1][2]
            img[i+1][j+1][2] = img[i][j+1][2]
            
    return img

def get_error_matrix(original, reinterp):
    error_matrix = original - reinterp
    error_matrix = error_matrix.astype('int8')
    error_matrix = np.divide(error_matrix, 2)
    error_matrix = np.around(error_matrix)
    error_matrix = np.clip(error_matrix, -3, 3)
    error_matrix = error_matrix.astype('int8')
    
    return error_matrix


def resample_bayer(img):
    # use only points which were measured exactly
    resampled = np.copy(img)
    for i in range(resampled.shape[0]):
        for j in range(resampled.shape[1]):
            if (i % 2 == 0 and j % 2 == 0) or (i % 2 == 1 and j % 2 == 1):
                # green was measured here
                resampled[i][j][0] = 0
                resampled[i][j][2] = 0
            elif i % 2 == 0 and j % 2 == 1:
                # blue was measured here
                resampled[i][j][0] = 0
                resampled[i][j][1] = 0
            else:
                # red was measured here
                resampled[i][j][1] = 0
                resampled[i][j][2] = 0
        
    return resampled
    
def get_intra_cooc_matrix(error_matrix):
    cooc = np.zeros((7,7,7))
    
    for i in range(0, error_matrix.shape[0], 2):
        for j in range(0, error_matrix.shape[0], 2):
            cooc[error_matrix[i][j][0]+3][error_matrix[i][j+1][0]+3][error_matrix[i+1][j+1][0]+3] += 1
    
    cooc = np.divide(cooc, (error_matrix.shape[0]/2)*(error_matrix.shape[1]/2))
    return cooc

def get_inter_cooc_matrix(error_matrix):
    cooc1 = np.zeros((7,7,7))
    cooc2 = np.zeros((7,7,7))
    
    for i in range(0, error_matrix.shape[0], 2):
        for j in range(0, error_matrix.shape[0], 2):
            cooc1[error_matrix[i][j][0]+3][error_matrix[i][j+1][0]+3][error_matrix[i][j+1][1]+3] += 1
            
    for i in range(1, error_matrix.shape[0], 2):
        for j in range(1, error_matrix.shape[0], 2):
            cooc2[error_matrix[i][j][0]+3][error_matrix[i-1][j][0]+3][error_matrix[i-1][j][1]+3] += 1
    
    cooc1 = np.divide(cooc1, (error_matrix.shape[0]/2)*(error_matrix.shape[1]/2))
    cooc2 = np.divide(cooc1, (error_matrix.shape[0]/2)*(error_matrix.shape[1]/2))
    
    return cooc1+cooc2
    

In [3]:
# read in test images

import cv2
import os
import math
from tqdm import tqdm_notebook as tqdm
DATA_DIR = '/home/jeff/Downloads/data/camera-model/'

test_dir = os.path.join(DATA_DIR, 'test')
X = None
for filename in tqdm(os.listdir(test_dir)):
    full_filename = os.path.join(test_dir, filename)
    img = cv2.imread(full_filename)
    crop = img[(math.floor(img.shape[0]/2)-256):(math.floor(img.shape[0]/2)+256),(math.floor(img.shape[1]/2)-256):(math.floor(img.shape[1]/2)+256)]
    resampled = resample_bayer(crop)
    reinterp1 = reinterp_nearest_neighbor(resampled)
    reinterp2 = reinterp_bilinear(resampled)
    error1 = get_error_matrix(crop, reinterp1)
    error2 = get_error_matrix(crop, reinterp2)
    cooc1_1 = get_intra_cooc_matrix(error1)
    cooc1_2 = get_inter_cooc_matrix(error1)
    cooc2_1 = get_intra_cooc_matrix(error2)
    cooc2_2 = get_intra_cooc_matrix(error2)

    img_features = np.append(filename, [cooc1_1, cooc1_2, cooc2_1, cooc2_2])
    img_features = np.array([img_features])
    if X is None:
        X = img_features
    else:
        X = np.append(X, img_features, axis=0)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [4]:
Y = X.copy()

In [7]:
import pandas as pd

df = pd.DataFrame(Y)
df.to_csv(os.path.join(DATA_DIR, 'test_features.csv'), header=False, index=False)

In [ ]:
X[0][0]